In [1]:
import sys

print(sys.version)


3.9.13 (main, Aug 25 2022, 23:26:10) 
[GCC 11.2.0]


In [2]:
import pandas as pd
from random import randint

import numpy as np
# from scipy import spatial
import random

In [3]:
data = pd.read_csv('./ratings.dat', sep='::', header = None)  # tab-separated data

/tmp/ipykernel_60616/2193334887.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv('./ratings.dat', sep='::', header = None)  # tab-separated data


In [4]:
data.head(100)

,0,1,2,3
0,1,122,5.0,838985046
1,1,185,5.0,838983525
2,1,231,5.0,838983392
3,1,292,5.0,838983421
4,1,316,5.0,838983392
...,...,...,...,...
95,4,377,3.0,844416892
96,4,380,3.0,844416656
97,4,410,5.0,844416936
98,4,420,3.0,844417004


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000054 entries, 0 to 10000053
Data columns (total 4 columns):
 #   Column  Dtype  
---  ------  -----  
 0   0       int64  
 1   1       int64  
 2   2       float64
 3   3       int64  
dtypes: float64(1), int64(3)
memory usage: 305.2 MB


In [6]:
data.columns=['User_ID','movie_ID','rating','timestamp']

In [7]:
data

,User_ID,movie_ID,rating,timestamp
0,1,122,5.0,838985046
1,1,185,5.0,838983525
2,1,231,5.0,838983392
3,1,292,5.0,838983421
4,1,316,5.0,838983392
...,...,...,...,...
10000049,71567,2107,1.0,912580553
10000050,71567,2126,2.0,912649143
10000051,71567,2294,5.0,912577968
10000052,71567,2338,2.0,912578016


In [8]:
data.drop('timestamp', axis=1, inplace=True)

In [9]:
data = data.head(3000)

In [10]:
data

,User_ID,movie_ID,rating
0,1,122,5.0
1,1,185,5.0
2,1,231,5.0
3,1,292,5.0
4,1,316,5.0
...,...,...,...
2995,34,542,4.0
2996,34,546,1.0
2997,34,547,3.0
2998,34,551,3.0


In [11]:
# Get unique User_IDs and movie_IDs
unique_users = data['User_ID'].unique()
unique_items = data['movie_ID'].unique()

In [12]:
# Number of users and items
n_users = len(unique_users)
n_items = len(unique_items)
print(n_users)
print(n_items)

28
1595


In [13]:

# Initialize R matrix with zeros
R = pd.DataFrame(0, index=unique_users, columns=unique_items, dtype=float)
RR = pd.DataFrame(0, index=unique_users, columns=unique_items, dtype=float)

# Fill R matrix based on the dataset
for index, row in data.iterrows():
    user_id = row['User_ID']
    item_id = row['movie_ID']
    # If there's a rating provided, set the corresponding entry to 1
    # if not pd.isnull(row['rating']):
    R.loc[user_id, item_id] = 1
    RR.loc[user_id, item_id] = row['rating']

In [14]:
R.reset_index(drop=True, inplace=True)
RR.reset_index(drop=True, inplace=True)

In [15]:
R.columns = range(len(R.columns))
RR.columns = range(len(RR.columns))

In [16]:
R = R.values
RR = RR.values

In [17]:
unique_users

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 17, 18,
       19, 22, 23, 24, 26, 27, 28, 29, 30, 33, 34])

In [18]:
R

array([[1., 1., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 1., ..., 1., 1., 1.]])

In [19]:
# Define the shape of the matrix
rows = R.shape[1]
cols = R.shape[1]

# Generate the zero matrix with values between 0 and 1
S = np.random.rand(rows, cols)

In [20]:
def generate_S_pu(num_clusters, rows):
    S_pu_list = []
    for _ in range(num_clusters):
        S_pu = np.random.rand(rows, rows)  # Generate zero matrix with values between 0 and 1
        S_pu_list.append(S_pu)
    return S_pu_list

# Define parameters
num_clusters = 3
# Generate random matrices for each cluster
S_pu_list = generate_S_pu(num_clusters, rows)
len(S_pu_list)

3

In [21]:
def constraint(matrix):
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            if i==j:
                matrix[i,j] = 0
    return matrix

In [22]:
S = constraint(S)
for pu in range(num_clusters):
    S_pu_list[pu] = constraint(S_pu_list[pu])

In [23]:
# Print the list of random matrices
for i, S_pu in enumerate(S_pu_list):
    print(f"S_pu matrix for cluster {i+1}:\n{S_pu}\n")

S_pu matrix for cluster 1:
[[0.         0.30008594 0.76215761 ... 0.22446301 0.50534682 0.88358674]
 [0.39017914 0.         0.29523915 ... 0.47315413 0.05424701 0.75034478]
 [0.80447706 0.83855226 0.         ... 0.47211965 0.93193169 0.48764637]
 ...
 [0.21082917 0.62087632 0.59079685 ... 0.         0.39423013 0.7204482 ]
 [0.42295141 0.74749897 0.72550575 ... 0.75702457 0.         0.35538253]
 [0.16260357 0.79708269 0.75819759 ... 0.11951576 0.37669781 0.        ]]

S_pu matrix for cluster 2:
[[0.         0.28685908 0.25026494 ... 0.51784227 0.04632213 0.92324459]
 [0.94978785 0.         0.77233281 ... 0.85921721 0.40258515 0.67921806]
 [0.89788561 0.99569591 0.         ... 0.27508316 0.5108785  0.89298468]
 ...
 [0.92692647 0.84180301 0.57465411 ... 0.         0.13172516 0.41584931]
 [0.08346356 0.2256164  0.71077626 ... 0.6664123  0.         0.25799104]
 [0.76101649 0.7925335  0.42894547 ... 0.7502486  0.89535226 0.        ]]

S_pu matrix for cluster 3:
[[0.         0.263959   0.560

In [24]:
def initial_clustering(n_clusters, users):
  random.shuffle(users)  # Shuffle the list randomly
  sublists = []
  size_sublist = len(users) //n_clusters  # Determine the size of each sublist

  # Split the shuffled list into n sublists
  for i in range(n_clusters):
      start = i * size_sublist
      end = (i + 1) * size_sublist if i < n_clusters - 1 else None
      sublists.append(users[start:end])
  return sublists

In [25]:
def compute_gu(pu,u):
  numerator = 0
  denomenator = 0
  for item in range(n_items):

    multiplier1 = 0
    for ind, Ruitem in enumerate(R[u,:]):
      if Ruitem == 1:
        multiplier1 += S[ind, item] - S_pu_list[pu][ind, item]
        denomenator += (S[ind, item] - S_pu_list[pu][ind, item])**2
    multiplier2 = R[u, item]
    for ind, Ruitem in enumerate(R[u,:]):
      if Ruitem == 1:
        multiplier2 -= S_pu_list[pu][ind, item]

    numerator += multiplier1*multiplier2
  return numerator/denomenator


In [26]:
def soft_thresholding(vect, lamb):
    for index, item in enumerate(vect):
        vect[index] = np.maximum(np.sign(item)*(np.abs(item)- lamb),0)
    return vect

In [27]:
def estimate_S(n_clusters, clusters, gu_users, S, S_pu_list, lambdag, betag):
    newS = np.zeros((S.shape[0], S.shape[1]))
    Rpu_list = []
    for cluster in clusters:
        Rpu = np.zeros((n_users, n_items))
        for user in cluster:
            for ind, item in enumerate(R[user, :]):
                if item == 1:
                    Rpu[user, ind] = 1
        Rpu_list.append(Rpu)
    g = gu_users.reshape(-1,1)
    gdash = 1-g
    A = np.multiply(g,R)
    B=[]
    for i in range(n_clusters):
        B.append(np.multiply(gdash, Rpu_list[i]))
    for item in range(n_items):
        ri  = R[:, item]
        summationofspui = np.zeros((n_users, 1))
        for j in range(n_clusters):
            summationofspui += B[j]@(S_pu_list[j][:, item]).reshape(-1,1)
        leftsideofSoftthresholding = ((A.T@ri).reshape(-1,1))-(A.T@summationofspui)
        rightsideofSofthreholding = lambdag
        Softthresholdingoutput = soft_thresholding(leftsideofSoftthresholding, rightsideofSofthreholding)
        newS[:,item] = (np.linalg.inv(betag*np.identity(n_items) + A.T@A)@Softthresholdingoutput).flatten()
    newS = constraint(newS)
    return newS

In [28]:
def estimate_Spu(n_clusters, clusters, gu_users, S, S_pu_list, betal, lambdal):
    newSpu = []
    for m in range(len(S_pu_list)):
        newSpu.append(np.zeros((S_pu_list[m].shape[0], S_pu_list[m].shape[1])))
    Rpu_list = []
    for cluster in clusters:
        Rpu = np.zeros((n_users, n_items))
        for user in cluster:
            for ind, item in enumerate(R[user, :]):
                if item == 1:
                    Rpu[user, ind] = 1
        Rpu_list.append(Rpu)
    g = gu_users.reshape(-1,1)
    gdash = 1-g
    A = np.multiply(g,R)
    B=[]
    for i in range(n_clusters):
        B.append(np.multiply(gdash, Rpu_list[i]))

    for item in range(n_items):
        for pu in range(n_clusters):
            ri  = R[:, item]
            si = S[:, item]
            leftsideofSoftthresholding = ((B[pu].T@ri) - (B[pu].T@A@si)).reshape((n_items, 1))
            summationofspui = np.zeros((n_users, 1))
            for j in range(n_clusters):
                if j != pu:
                    summationofspui += ((B[j])@(S_pu_list[j][:, item].reshape((n_items,1)))).reshape((n_users, 1))
            leftsideofSoftthresholding -= B[pu].T@summationofspui
            rightsideofSofthreholding = lambdal
            Softthresholdingoutput = soft_thresholding(leftsideofSoftthresholding, rightsideofSofthreholding)
            newSpu[pu][:,item] = (np.linalg.inv(betal*np.identity(n_items) + B[pu].T@B[pu])@(Softthresholdingoutput).flatten())
    for pu in range(num_clusters):
        newSpu[pu] = constraint(newSpu[pu])
    return newSpu

**GLSLIM**

In [29]:
def glslim(S, S_pu_list):
  gu_users = np.array([0.5 for u in range(n_users)])
  n_clusters = 3
  clusters = initial_clustering(n_clusters, list(range(n_users)))
  print("initial cluster: ", clusters)
  while True:
    count = 0
    S = estimate_S(n_clusters, clusters, gu_users, S, S_pu_list, 0.1, 0.2)
    S_pu_list =estimate_Spu(n_clusters, clusters, gu_users, S, S_pu_list, 0.2, 0.1)
    for u in list(range(n_users)):
      storetrainingerror = []

      for pu in range(n_clusters):
        gu = compute_gu(pu, u)

        modR = sum(R[u,:])
        training_error = 0
        for i, Ruitem in enumerate(R[u,:]):
          if Ruitem == 1:
            diff = RR[u, i]
            rui = 0
            for l, Ruitem in enumerate(R[u,:]):
              if Ruitem == 1:
                rui += gu*S[l,i] + (1-gu)*S_pu_list[pu][l,i]
            diff -= rui
            training_error+= diff**2
        training_error /= modR
        training_error = np.sqrt(training_error)
        storetrainingerror.append(training_error)
      argminpu = np.argmin(storetrainingerror)
      if u not in clusters[argminpu]:
        for k in range(n_clusters):
          if u in clusters[k]:
            clusters[k].remove(u)
        clusters[argminpu].append(u)
        count +=1
      gu_users[u] = gu
    print(count/n_users)
    if count/n_users <= 0.05:
      break
  return clusters, gu_users, S, S_pu_list

In [ ]:
clusters, gu_users, S, S_pu_list = glslim(S, S_pu_list)

initial cluster:  [[1, 2, 6, 26, 4, 22, 14, 20, 5], [3, 13, 9, 21, 24, 19, 8, 0, 27], [16, 25, 12, 23, 18, 10, 17, 11, 15, 7]]
0.10714285714285714
0.35714285714285715
0.14285714285714285
0.14285714285714285
0.17857142857142858
0.07142857142857142
0.10714285714285714
0.21428571428571427
0.17857142857142858
0.25
0.17857142857142858
0.10714285714285714


In [ ]:
clusters

In [ ]:
# Sample NumPy matrix
matrix = np.copy(R)

# Initialize list to store random indices
random_indices = []

# Iterate over each row
for row in matrix:
    # Get indices of non-zero elements in the row
    nonzero_indices = np.nonzero(row)[0]
    if len(nonzero_indices) > 0:
        # Choose one random index from the non-zero indices
        random_index = np.random.choice(nonzero_indices)
        random_indices.append(random_index)

print(random_indices)

In [ ]:
print(matrix[0,19])

In [ ]:

for ind, row in enumerate(matrix):
  row[random_indices[ind]] = 0

In [ ]:
#run glslim on this data, get S and Spu matrices
n_clusters = 3
user_non_rated_item_predicted_ratings = []
for u in list(range(n_users)):
  user_store_for_each_item = []
  for i, Ruitem in enumerate(matrix[u,:]):
    rui = 0
    if Ruitem == 0.0:
      
      user_pu = 0
      for pu in range(n_clusters):
          if u in clusters[pu]:
            user_pu = pu
      for l, Ruitem in enumerate(R[u,:]):
        if Ruitem == 1:
          rui += gu_users[u]*S[l,i] + (1-gu_users[u])*S_pu_list[user_pu][l,i]
    user_store_for_each_item.append((i, rui))
  user_store_for_each_item = sorted(user_store_for_each_item, key=lambda x: x[0], reverse=True)
  user_non_rated_item_predicted_ratings.append(user_store_for_each_item)

In [ ]:
hits = 0
N = 20
for user in range(n_users):
    recommended_items = user_non_rated_item_predicted_ratings[user][:N]
    first_elements = [t[0] for t in recommended_items]
    if random_indices[user] in first_elements:
        hits += 1

hit_rate = hits / n_users
print(hit_rate)

In [ ]:
#For 4 clusters

In [ ]:
def generate_S_pu(num_clusters, rows):
    S_pu_list = []
    for _ in range(num_clusters):
        S_pu = np.random.rand(rows, rows)  # Generate zero matrix with values between 0 and 1
        S_pu_list.append(S_pu)
    return S_pu_list

# Define parameters
num_clusters = 4
# Generate random matrices for each cluster
S_pu_list = generate_S_pu(num_clusters, rows)
len(S_pu_list)

In [ ]:
def constraint(matrix):
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            if i==j:
                matrix[i,j] = 0
    return matrix

In [ ]:
S = constraint(S)
for pu in range(num_clusters):
    S_pu_list[pu] = constraint(S_pu_list[pu])

In [ ]:
def estimate_Spu(n_clusters, clusters, gu_users, S, S_pu_list, betal, lambdal):
    newSpu = []
    for m in range(len(S_pu_list)):
        newSpu.append(np.zeros((S_pu_list[m].shape[0], S_pu_list[m].shape[1])))
    Rpu_list = []
    for cluster in clusters:
        Rpu = np.zeros((n_users, n_items))
        for user in cluster:
            for ind, item in enumerate(R[user, :]):
                if item == 1:
                    Rpu[user, ind] = 1
        Rpu_list.append(Rpu)
    g = gu_users.reshape(-1,1)
    gdash = 1-g
    A = np.multiply(g,R)
    B=[]
    for i in range(n_clusters):
        B.append(np.multiply(gdash, Rpu_list[i]))

    for item in range(n_items):
        for pu in range(n_clusters):
            ri  = R[:, item]
            si = S[:, item]
            leftsideofSoftthresholding = ((B[pu].T@ri) - (B[pu].T@A@si)).reshape((n_items, 1))
            summationofspui = np.zeros((n_users, 1))
            for j in range(n_clusters):
                if j != pu:
                    summationofspui += ((B[j])@(S_pu_list[j][:, item].reshape((n_items,1)))).reshape((n_users, 1))
            leftsideofSoftthresholding -= B[pu].T@summationofspui
            rightsideofSofthreholding = lambdal
            Softthresholdingoutput = soft_thresholding(leftsideofSoftthresholding, rightsideofSofthreholding)
            newSpu[pu][:,item] = (np.linalg.inv(betal*np.identity(n_items) + B[pu].T@B[pu])@(Softthresholdingoutput).flatten())
    for pu in range(num_clusters):
        newSpu[pu] = constraint(newSpu[pu])
    return newSpu

In [ ]:
def glslim(S, S_pu_list):
  gu_users = np.array([0.5 for u in range(n_users)])
  n_clusters = 3
  clusters = initial_clustering(n_clusters, list(range(n_users)))
  print("initial cluster: ", clusters)
  while True:
    count = 0
    S = estimate_S(n_clusters, clusters, gu_users, S, S_pu_list, 0.1, 0.2)
    S_pu_list =estimate_Spu(n_clusters, clusters, gu_users, S, S_pu_list, 0.2, 0.1)
    for u in list(range(n_users)):
      storetrainingerror = []

      for pu in range(n_clusters):
        gu = compute_gu(pu, u)

        modR = sum(R[u,:])
        training_error = 0
        for i, Ruitem in enumerate(R[u,:]):
          if Ruitem == 1:
            diff = RR[u, i]
            rui = 0
            for l, Ruitem in enumerate(R[u,:]):
              if Ruitem == 1:
                rui += gu*S[l,i] + (1-gu)*S_pu_list[pu][l,i]
            diff -= rui
            training_error+= diff**2
        training_error /= modR
        training_error = np.sqrt(training_error)
        storetrainingerror.append(training_error)
      argminpu = np.argmin(storetrainingerror)
      if u not in clusters[argminpu]:
        for k in range(n_clusters):
          if u in clusters[k]:
            clusters[k].remove(u)
        clusters[argminpu].append(u)
        count +=1
      gu_users[u] = gu
    print(count/n_users)
    if count/n_users <= 0.05:
      break
  return clusters, gu_users, S, S_pu_list

In [ ]:
hits = 0
N = 30
for user in range(n_users):
    recommended_items = user_non_rated_item_predicted_ratings[user][:N]
    first_elements = [t[0] for t in recommended_items]
    if random_indices[user] in first_elements:
        hits += 1

hit_rate = hits / n_users
print(hit_rate)

In [ ]:
#For 5 clusters

In [ ]:
def generate_S_pu(num_clusters, rows):
    S_pu_list = []
    for _ in range(num_clusters):
        S_pu = np.random.rand(rows, rows)  # Generate zero matrix with values between 0 and 1
        S_pu_list.append(S_pu)
    return S_pu_list

# Define parameters
num_clusters = 5
# Generate random matrices for each cluster
S_pu_list = generate_S_pu(num_clusters, rows)
len(S_pu_list)

In [ ]:
def constraint(matrix):
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            if i==j:
                matrix[i,j] = 0
    return matrix

In [ ]:
S = constraint(S)
for pu in range(num_clusters):
    S_pu_list[pu] = constraint(S_pu_list[pu])

In [ ]:
def estimate_Spu(n_clusters, clusters, gu_users, S, S_pu_list, betal, lambdal):
    newSpu = []
    for m in range(len(S_pu_list)):
        newSpu.append(np.zeros((S_pu_list[m].shape[0], S_pu_list[m].shape[1])))
    Rpu_list = []
    for cluster in clusters:
        Rpu = np.zeros((n_users, n_items))
        for user in cluster:
            for ind, item in enumerate(R[user, :]):
                if item == 1:
                    Rpu[user, ind] = 1
        Rpu_list.append(Rpu)
    g = gu_users.reshape(-1,1)
    gdash = 1-g
    A = np.multiply(g,R)
    B=[]
    for i in range(n_clusters):
        B.append(np.multiply(gdash, Rpu_list[i]))

    for item in range(n_items):
        for pu in range(n_clusters):
            ri  = R[:, item]
            si = S[:, item]
            leftsideofSoftthresholding = ((B[pu].T@ri) - (B[pu].T@A@si)).reshape((n_items, 1))
            summationofspui = np.zeros((n_users, 1))
            for j in range(n_clusters):
                if j != pu:
                    summationofspui += ((B[j])@(S_pu_list[j][:, item].reshape((n_items,1)))).reshape((n_users, 1))
            leftsideofSoftthresholding -= B[pu].T@summationofspui
            rightsideofSofthreholding = lambdal
            Softthresholdingoutput = soft_thresholding(leftsideofSoftthresholding, rightsideofSofthreholding)
            newSpu[pu][:,item] = (np.linalg.inv(betal*np.identity(n_items) + B[pu].T@B[pu])@(Softthresholdingoutput).flatten())
    for pu in range(num_clusters):
        newSpu[pu] = constraint(newSpu[pu])
    return newSpu

In [ ]:
def glslim(S, S_pu_list):
  gu_users = np.array([0.5 for u in range(n_users)])
  n_clusters = 3
  clusters = initial_clustering(n_clusters, list(range(n_users)))
  print("initial cluster: ", clusters)
  while True:
    count = 0
    S = estimate_S(n_clusters, clusters, gu_users, S, S_pu_list, 0.1, 0.2)
    S_pu_list =estimate_Spu(n_clusters, clusters, gu_users, S, S_pu_list, 0.2, 0.1)
    for u in list(range(n_users)):
      storetrainingerror = []

      for pu in range(n_clusters):
        gu = compute_gu(pu, u)

        modR = sum(R[u,:])
        training_error = 0
        for i, Ruitem in enumerate(R[u,:]):
          if Ruitem == 1:
            diff = RR[u, i]
            rui = 0
            for l, Ruitem in enumerate(R[u,:]):
              if Ruitem == 1:
                rui += gu*S[l,i] + (1-gu)*S_pu_list[pu][l,i]
            diff -= rui
            training_error+= diff**2
        training_error /= modR
        training_error = np.sqrt(training_error)
        storetrainingerror.append(training_error)
      argminpu = np.argmin(storetrainingerror)
      if u not in clusters[argminpu]:
        for k in range(n_clusters):
          if u in clusters[k]:
            clusters[k].remove(u)
        clusters[argminpu].append(u)
        count +=1
      gu_users[u] = gu
    print(count/n_users)
    if count/n_users <= 0.05:
      break
  return clusters, gu_users, S, S_pu_list

In [ ]:
clusters, gu_users, S, S_pu_list = glslim(S, S_pu_list)

In [ ]:
clusters

In [ ]:
# Sample NumPy matrix
matrix = np.copy(R)

# Initialize list to store random indices
random_indices = []

# Iterate over each row
for row in matrix:
    # Get indices of non-zero elements in the row
    nonzero_indices = np.nonzero(row)[0]
    if len(nonzero_indices) > 0:
        # Choose one random index from the non-zero indices
        random_index = np.random.choice(nonzero_indices)
        random_indices.append(random_index)

print(random_indices)

In [ ]:
for ind, row in enumerate(matrix):
  row[random_indices[ind]] = 0

In [ ]:
#run glslim on this data, get S and Spu matrices
n_clusters = 4
user_non_rated_item_predicted_ratings = []
for u in list(range(n_users)):
  user_store_for_each_item = []
  for i, Ruitem in enumerate(matrix[u,:]):
    rui = 0
    if Ruitem == 0.0:
      
      user_pu = 0
      for pu in range(n_clusters):
          if u in clusters[pu]:
            user_pu = pu
      for l, Ruitem in enumerate(R[u,:]):
        if Ruitem == 1:
          rui += gu_users[u]*S[l,i] + (1-gu_users[u])*S_pu_list[user_pu][l,i]
    user_store_for_each_item.append((i, rui))
  user_store_for_each_item = sorted(user_store_for_each_item, key=lambda x: x[0], reverse=True)
  user_non_rated_item_predicted_ratings.append(user_store_for_each_item)

In [ ]:
#run glslim on this data, get S and Spu matrices
n_clusters = 5
user_non_rated_item_predicted_ratings = []
for u in list(range(n_users)):
  user_store_for_each_item = []
  for i, Ruitem in enumerate(matrix[u,:]):
    rui = 0
    if Ruitem == 0.0:
      
      user_pu = 0
      for pu in range(n_clusters):
          if u in clusters[pu]:
            user_pu = pu
      for l, Ruitem in enumerate(R[u,:]):
        if Ruitem == 1:
          rui += gu_users[u]*S[l,i] + (1-gu_users[u])*S_pu_list[user_pu][l,i]
    user_store_for_each_item.append((i, rui))
  user_store_for_each_item = sorted(user_store_for_each_item, key=lambda x: x[0], reverse=True)
  user_non_rated_item_predicted_ratings.append(user_store_for_each_item)

In [ ]:
hits = 0
N = 30
for user in range(n_users):
    recommended_items = user_non_rated_item_predicted_ratings[user][:N]
    first_elements = [t[0] for t in recommended_items]
    if random_indices[user] in first_elements:
        hits += 1

hit_rate = hits / n_users
print(hit_rate)

In [213]:
#LSLIM

In [214]:
no_of_cluster=3

In [215]:
def estimate_Spu(n_clusters, clusters, gu_users,S_pu_list, betal, lambdal):
    newSpu = []
    for m in range(len(S_pu_list)):
        newSpu.append(np.zeros((S_pu_list[m].shape[0], S_pu_list[m].shape[1])))
    Rpu_list = []
    for cluster in clusters:
        Rpu = np.zeros((n_users, n_items))
        for user in cluster:
            for ind, item in enumerate(R[user, :]):
                if item == 1:
                    Rpu[user, ind] = 1
        Rpu_list.append(Rpu)
    g = gu_users.reshape(-1,1)
    gdash = 1-g
    A = np.multiply(g,R)
    B=[]
    for i in range(n_clusters):
        B.append(Rpu_list[i])

    for item in range(n_items):
        for pu in range(n_clusters):
            ri  = Rpu_list[pu][:, item]
            si = S[:, item]
            leftsideofSoftthresholding = ((B[pu].T@ri)).reshape((n_items, 1))
            summationofspui = np.zeros((n_users, 1))
            for j in range(n_clusters):
                if j != pu:
                    summationofspui += ((B[j])@(S_pu_list[j][:, item].reshape((n_items,1)))).reshape((n_users, 1))
            leftsideofSoftthresholding -= B[pu].T@summationofspui
            rightsideofSofthreholding = lambdal
            Softthresholdingoutput = soft_thresholding(leftsideofSoftthresholding, rightsideofSofthreholding)
            newSpu[pu][:,item] = (np.linalg.inv(betal*np.identity(n_items) + B[pu].T@B[pu])@(Softthresholdingoutput).flatten())
    for pu in range(num_clusters):
        newSpu[pu] = constraint(newSpu[pu])
    return newSpu

In [ ]:
clusters, gu_users, S, S_pu_list = glslim(S, S_pu_list)

In [ ]:
clusters

In [ ]:
# Sample NumPy matrix
matrix = np.copy(R)

# Initialize list to store random indices
random_indices = []

# Iterate over each row
for row in matrix:
    # Get indices of non-zero elements in the row
    nonzero_indices = np.nonzero(row)[0]
    if len(nonzero_indices) > 0:
        # Choose one random index from the non-zero indices
        random_index = np.random.choice(nonzero_indices)
        random_indices.append(random_index)

print(random_indices)

In [ ]:
for ind, row in enumerate(matrix):
  row[random_indices[ind]] = 0

In [216]:
def lslim(S_pu_list):
  gu_users = np.array([0.5 for u in range(n_users)])
  n_clusters = 3
  clusters = initial_clustering(n_clusters, list(range(n_users)))
  print("initial cluster: ", clusters)
  while True:
    count = 0
    S_pu_list =estimate_Spu(n_clusters, clusters, gu_users,S_pu_list, 0.2, 0.1)
    for u in list(range(n_users)):
      storetrainingerror = []

      for pu in range(n_clusters):
        gu = compute_gu(pu, u)

        modR = sum(R[u,:])
        training_error = 0
        for i, Ruitem in enumerate(R[u,:]):
          if Ruitem == 1:
            diff = RR[u, i]
            rui = 0
            for l, Ruitem in enumerate(R[u,:]):
              if Ruitem == 1:
                rui += S_pu_list[pu][l,i]
            diff -= rui
            training_error+= diff**2
        training_error /= modR
        training_error = np.sqrt(training_error)
        storetrainingerror.append(training_error)
      argminpu = np.argmin(storetrainingerror)
      if u not in clusters[argminpu]:
        for k in range(n_clusters):
          if u in clusters[k]:
            clusters[k].remove(u)
        clusters[argminpu].append(u)
        count +=1
      gu_users[u] = gu
    print(count/n_users)
    if count/n_users <= 0.05:
      break
  return clusters, S_pu_list

In [217]:
clusters, S_pu_list = lslim(S_pu_list)

initial cluster:  [[2, 7], [1, 6], [4, 0, 5, 3]]


/tmp/ipykernel_2757484/4209350880.py:17: RuntimeWarning: invalid value encountered in double_scalars
  return numerator/denomenator


0.0


In [218]:
clusters

[[2, 7], [1, 6], [4, 0, 5, 3]]

In [219]:
n_clusters=3
Rpu_list = []
for cluster in clusters:
    Rpu = np.zeros((n_users, n_items))
    for user in cluster:
        for ind, item in enumerate(R[user, :]):
            if item == 1:
                Rpu[user, ind] = 1
    Rpu_list.append(Rpu)

In [220]:
# Sample NumPy matrix
matrix = np.copy(R)

# Initialize list to store random indices
random_indices = []

# Iterate over each row
for row in matrix:
    # Get indices of non-zero elements in the row
    nonzero_indices = np.nonzero(row)[0]
    if len(nonzero_indices) > 0:
        # Choose one random index from the non-zero indices
        random_index = np.random.choice(nonzero_indices)
        random_indices.append(random_index)

print(random_indices)

[10, 30, 50, 77, 106, 194, 44, 545]


In [221]:

for ind, row in enumerate(matrix):
  row[random_indices[ind]] = 0

In [222]:
#run glslim on this data, get S and Spu matrices
n_clusters = 3
user_non_rated_item_predicted_ratings = []
for u in list(range(n_users)):
  user_store_for_each_item = []
  for i, Ruitem in enumerate(matrix[u,:]):
    rui = 0
    if Ruitem == 0.0:
      
      user_pu = 0
      for pu in range(n_clusters):
          if u in clusters[pu]:
            user_pu = pu
      for l, Ruitem in enumerate(R[u,:]):
        if Ruitem == 1:
          rui += S_pu_list[user_pu][l,i]
    user_store_for_each_item.append((i, rui))
  user_store_for_each_item = sorted(user_store_for_each_item, key=lambda x: x[0], reverse=True)
  user_non_rated_item_predicted_ratings.append(user_store_for_each_item)

In [227]:
hits = 0
N = 20
for user in range(n_users):
    recommended_items = [t[0] for t in user_non_rated_item_predicted_ratings[user][:N]]
    if random_indices[user] in recommended_items:
        hits += 1

hit_rate = hits / n_users

print("Hit rate:", hit_rate)



Hit rate: 0.0
